# RecBERT3LM

可用的中文预训练参数：[`bert-base`](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip),[`roberta-wwm-ext-base`](https://drive.google.com/uc?export=download&id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt),[`roberta-wwm-ext-large`](https://drive.google.com/uc?export=download&id=1dtad0FFzG11CBsawu8hvwwzU2R0FDI94),[`macbert-base`](https://drive.google.com/uc?export=download&id=1aV69OhYzIwj_hn-kO1RiBa-m8QAusQ5b),[`macbert-large`](https://drive.google.com/uc?export=download&id=1lWYxnk1EqTA2Q20_IShxBrCPc5VSDCkT)

In [1]:
import uf

print(uf.__version__)

v2.5.20


In [2]:
model = uf.RecBERT3LM("../../ref/bert_config.json", "../../ref/vocab.txt", max_seq_length=10)
print(model)

uf.RecBERT3LM(
    config_file="../../ref/bert_config.json",
    vocab_file="../../ref/vocab.txt",
    max_seq_length=10,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    add_prob=0.1,
    rep_prob=0.1,
    do_lower_case=True,
    truncate_method="LIFO",
)


In [3]:
X = ["天亮以前说再见", "笑着泪流满面", "去迎接应该你的", "更好的明天", "他想知道那是谁", "为何总沉默寡言", "人群中也算抢眼", "抢眼的孤独难免"]

# 训练

In [4]:
model.fit(X, total_steps=20)

/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  query_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:246: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  key_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:256: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  value_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:380: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Ple

INFO:tensorflow:Build graph with 333,374,994 parameters (among which 111,124,998 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 8 samples (step 0 -> 20)
INFO:tensorflow:step 1, add_accuracy 0.4375, add_loss 0.688775, rep_accuracy 0.6875, rep_loss 0.537048, cls_accuracy 0.7500, cls_loss 0.606890, 0.13 steps/sec, 1.02 examples/sec
INFO:tensorflow:step 2, add_accuracy 0.9219, add_loss 0.280191, rep_accuracy 0.9375, rep_loss 0.214980, cls_accuracy 0.7500, cls_loss 0.934923, 0.64 steps/sec, 5.14 examples/sec
INFO:tensorflow:step 3, add_accuracy 0.9219, add_loss 0.297513, rep_accuracy 0.9375, rep_loss 0.203011, cls_accuracy 0.7500, cls_loss 0.406371, 0.65 steps/sec, 5.18 examples/sec
INFO:tensorflow:step 4, add_accuracy 0.9219, add_loss 0.230842, rep_accuracy 0.9375, rep_loss 0.167780, cls_accuracy 0.2500, cls_loss 1.359688, 0.61 steps/sec, 4.86 examples/sec
INFO:tensorflow:step 5, add_accuracy 0.9219, add_l

# 推理

In [5]:
X = [
    "".join(
        token for token 
        in model.tokenizer.convert_ids_to_tokens(model.data["input_ids"][i])
        if token not in ("[CLS]", "[SEP]", "[PAD]")
    )
    for i in range(len(model.data["input_ids"]))
]
# X = [
#     "他想知道那是谁道", "为何总沉默寡", 
#     "群人面群中也抢面", "抢孤为独道难免",
# ]           # 人为增加一些错误，看模型是否能识别并纠正 (与训练阶段采样不同，因此此case预测错误正常)
model.predict(X)

INFO:tensorflow:Running inference on 8 samples
INFO:tensorflow:process 100.0%, 6.30 examples/sec


{'lm_preds': ['他想{rep:知}道那是谁',
  '{add:满}好的明天',
  '天亮以前说再{rep:见}',
  '人群中也算抢眼',
  '为何{rep:总}沉默{add:寡}言',
  '笑着泪流{add:满}面',
  '抢{add:眼}的孤{rep:独}{add:难}免',
  '去迎接应该你的'],
 'cls_probs': array([[1.7809802e-03, 9.9821895e-01],
        [5.3508801e-04, 9.9946493e-01],
        [2.3295372e-03, 9.9767047e-01],
        [9.9092346e-01, 9.0765636e-03],
        [1.8411953e-03, 9.9815875e-01],
        [1.7286688e-03, 9.9827135e-01],
        [2.6518737e-03, 9.9734807e-01],
        [9.9036998e-01, 9.6300319e-03]], dtype=float32),
 'cls_preds': [1, 1, 1, 0, 1, 1, 1, 0]}

# 训练样本生成

In [6]:
X = ["天亮以前说再见", "笑着泪流满面", "去迎接应该你的", "更好的明天", "他想知道那是谁", "为何总沉默寡言", "人群中也算抢眼", "抢眼的孤独难免"]
data = model.convert(X, is_training=True)
for i in range(len(data["input_ids"])):
    print("input_tokens: %s" % (model.tokenizer.convert_ids_to_tokens(data["input_ids"][i])))
    print("add_label_ids: %s" % (model.tokenizer.convert_ids_to_tokens(data["add_label_ids"][i])))
    print("rep_label_ids: %s" % (model.tokenizer.convert_ids_to_tokens(data["rep_label_ids"][i])))
    print("")


input_tokens: ['[CLS]', '亮', '天', '前', '说', '再', '见', '[SEP]', '[PAD]', '[PAD]']
add_label_ids: ['天', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
rep_label_ids: ['[PAD]', '[PAD]', '以', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

input_tokens: ['[CLS]', '笑', '知', '泪', '流', '满', '面', '[SEP]', '[PAD]', '[PAD]']
add_label_ids: ['[PAD]', '着', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
rep_label_ids: ['[PAD]', '[PAD]', '[Soft Prompt]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

input_tokens: ['[CLS]', '去', '眼', '迎', '接', '该', '你', '的', '[SEP]', '[PAD]']
add_label_ids: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '应', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
rep_label_ids: ['[PAD]', '[PAD]', '[Soft Prompt]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

input_tokens: ['[CLS]', '更', '好', '的', '明', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
add_label_ids: ['[PAD]', '[PAD]',